In [86]:
import Pkg
Pkg.activate("../../../FinancialPlanner")

  Activating project at `c:\Users\matsz\programowanie\Optymalizacja_portfela\FinancialPlanner`


In [95]:
using Revise
using LinearAlgebra
using Distributions
using Random
using StatsPlots
using PDMats
using StatsBase
using Base.Iterators
using DataFrames, TimeSeries, XLSX
using PrettyTables

In [96]:
includet("../VARs/utils.jl")
includet("../VARs/VAR_OLS.jl")

In [104]:
df = DataFrame(XLSX.readtable("../../data/usa_data.xlsx", "Data", infer_eltypes=true))
data_source = collapse(TimeArray(df; timestamp = :Date), Dates.quarter, last)

292×12 TimeArray{Any, 2, Date, Matrix{Any}} 1951-12-01 to 2024-07-01
┌────────────┬──────────┬───────────┬─────────────┬──────────┬────────┬─────────
│            │ Yeld_10Y │ Price_10Y │ Stock_Price │ Dividend │ E      │ CAPE   ⋯
├────────────┼──────────┼───────────┼─────────────┼──────────┼────────┼─────────
│ 1951-12-01 │  0.02513 │   104.098 │       23.77 │  1.41333 │   2.44 │ 12.527 ⋯
│ 1952-03-01 │  0.02505 │   104.824 │       24.37 │     1.43 │    2.4 │ 12.242 ⋯
│ 1952-06-01 │  0.02415 │   106.295 │       24.96 │     1.45 │   2.34 │ 12.669 ⋯
│ 1952-09-01 │  0.02685 │   104.468 │       24.54 │  1.43667 │   2.36 │ 12.131 ⋯
│ 1952-12-01 │  0.02547 │   106.402 │       26.57 │     1.41 │    2.4 │ 13.010 ⋯
│ 1953-03-01 │  0.02735 │   105.373 │       25.29 │  1.41333 │   2.43 │ 12.163 ⋯
│ 1953-06-01 │  0.02987 │    103.86 │       24.14 │     1.42 │   2.51 │ 11.750 ⋯
│ 1953-09-01 │  0.02636 │    107.81 │       23.35 │     1.43 │   2.55 │ 11.391 ⋯
│     ⋮      │    ⋮     │     ⋮     │   

In [ ]:
freq = 4
returns = percentchange(data_source[:Total_Stock, :Price_10Y], :log)
cpi = percentchange(data_source[:CPI], :log)

real_t_bill = log1p.(data_source[:T_Bill])./freq .- cpi
excess_returns = returns .- log1p.(data_source[:T_Bill]./freq)
dy = log.(data_source[:Dividend]) .- log.(data_source[:Stock_Price])

term = data_source[:Yeld_10Y] .- data_source[:T_Bill] 

data = merge(excess_returns, real_t_bill, dy, term, log1p.(data_source[:T_Bill]))

start = Date(1955, 01,01)
end_d = Date(2024, 12, 31)
s_data = to(from(data, start),end_d)

display(returns_summarystats(s_data, freq))

MethodError: MethodError: no method matching log(::String)
The function `log` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  log(!Matched::ComplexF16)
   @ Base math.jl:1512
  log(!Matched::Float16)
   @ Base math.jl:1511
  log(!Matched::BigFloat)
   @ Base mpfr.jl:786
  ...


In [ ]:
assets_names = colnames(s_data)
T, n_assets = size(s_data)
model = VARModel(s_data)
model_summary(model)

,const,Total_Stock_T_Bill,Price_10Y_T_Bill,T_Bill_CPI,Dividend_Stock_Price,Yeld_10Y_T_Bill,T_Bill,std
Total_Stock_T_Bill,0.1355,0.1217,0.1308,1.1797,0.0277,-0.1808,-0.5434,0.067
Price_10Y_T_Bill,-0.0343,-0.1262,-0.0605,0.9065,-0.0068,0.8016,0.0603,0.0399
T_Bill_CPI,-0.0041,-0.006,0.0021,0.3007,-0.0003,0.072,0.083,0.0071
Dividend_Stock_Price,-0.1046,-0.107,-0.1413,-1.4515,0.9757,0.107,0.3704,0.0693
Yeld_10Y_T_Bill,0.0031,-0.0036,0.0151,-0.0251,0.0004,0.8235,0.014,0.0071
T_Bill,0.0015,0.0188,-0.0061,-0.0897,0.0005,0.0828,0.9791,0.0083


,Total_Stock_T_Bill,Price_10Y_T_Bill,T_Bill_CPI,Dividend_Stock_Price,Yeld_10Y_T_Bill,T_Bill
Total_Stock_T_Bill,1.0,0.14,0.05,-0.97,0.1,-0.19
Price_10Y_T_Bill,0.14,1.0,0.17,-0.12,-0.03,-0.6
T_Bill_CPI,0.05,0.17,1.0,-0.07,-0.14,0.01
Dividend_Stock_Price,-0.97,-0.12,-0.07,1.0,-0.1,0.18
Yeld_10Y_T_Bill,0.1,-0.03,-0.14,-0.1,1.0,-0.77
T_Bill,-0.19,-0.6,0.01,0.18,-0.77,1.0


In [92]:
scenarios = simulate(model, 25*freq, 10_000)
periods = [1,5, 15, 25]
ret_in_years = cum_returns_in_periods(scenarios, periods, freq, true)
print_scenarios_summary(ret_in_years, assets_names, string.(periods))

for a in 1:n_assets
    print_scenarios_percentiles(ret_in_years[a, :, :], [.01, 0.025, .05, .25, .5, .75, .95, .975, .99], string.(periods), string.(assets_names[a]))
end  

,Total_Stock_T_Bill,Price_10Y_T_Bill,T_Bill_CPI,Dividend_Stock_Price,Yeld_10Y_T_Bill,T_Bill
1,-0.0318,0.0085,0.0196,-17.2697,-0.0229,0.1827
5,-0.0064,0.0108,0.0056,-16.9114,0.0217,0.1427
15,0.0191,0.0121,0.002,-16.4346,0.0383,0.1203
25,0.0279,0.0113,0.0015,-16.1834,0.0418,0.1174


,Total_Stock_T_Bill,Price_10Y_T_Bill,T_Bill_CPI,Dividend_Stock_Price,Yeld_10Y_T_Bill,T_Bill
1,0.1258,0.0667,0.015,0.2739,0.0229,0.0289
5,0.0587,0.0311,0.0088,0.6776,0.0268,0.0548
15,0.0316,0.014,0.0077,0.9624,0.0177,0.066
25,0.0236,0.0092,0.0076,1.032,0.0139,0.0672


,Total_Stock_T_Bill,Price_10Y_T_Bill,T_Bill_CPI,Dividend_Stock_Price,Yeld_10Y_T_Bill,T_Bill
1,0.0349,0.0327,0.0076,-0.0617,-0.0329,-0.0059
5,0.0145,-0.0533,-0.0027,0.0066,0.0064,0.0439
15,0.0149,0.0111,0.0061,-0.0292,0.0495,-0.0513
25,0.034,0.0275,-0.0526,-0.0239,0.0712,-0.0719


,Total_Stock_T_Bill,Price_10Y_T_Bill,T_Bill_CPI,Dividend_Stock_Price,Yeld_10Y_T_Bill,T_Bill
1,-0.0849,0.0554,0.04,-0.0476,-0.0449,0.0344
5,0.0332,0.0326,-0.0177,-0.0102,0.0185,0.074
15,-0.0696,0.0483,0.0183,-0.0854,0.0382,-0.0283
25,0.03,-0.0096,-0.0209,-0.0525,-0.0159,-0.0248


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,-0.3235,-0.2738,-0.2379,-0.1164,-0.0325,0.053,0.1781,0.2192,0.2616
5,-0.1435,-0.1207,-0.1026,-0.0464,-0.0063,0.033,0.0898,0.1084,0.1323
15,-0.054,-0.0426,-0.0326,-0.0026,0.0191,0.0405,0.0717,0.0818,0.0915
25,-0.0258,-0.0177,-0.0106,0.0119,0.0278,0.0435,0.0674,0.075,0.0835


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,-0.1481,-0.1207,-0.0996,-0.0362,0.0084,0.053,0.1195,0.1413,0.1651
5,-0.0619,-0.0497,-0.0413,-0.0101,0.0114,0.0318,0.0615,0.0712,0.0824
15,-0.0206,-0.0154,-0.0109,0.0027,0.0123,0.0216,0.0352,0.0396,0.0452
25,-0.0098,-0.0065,-0.0037,0.005,0.0113,0.0174,0.0265,0.0295,0.0328


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,-0.0156,-0.0099,-0.0051,0.0093,0.0196,0.0295,0.0444,0.0492,0.0548
5,-0.015,-0.0117,-0.009,-0.0004,0.0056,0.0116,0.0202,0.023,0.0259
15,-0.016,-0.0131,-0.0106,-0.0031,0.002,0.0072,0.0148,0.0171,0.0199
25,-0.0162,-0.0136,-0.0112,-0.0037,0.0015,0.0067,0.0137,0.016,0.019


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,-17.918,-17.8107,-17.7263,-17.4548,-17.2631,-17.0827,-16.8241,-16.7496,-16.645
5,-18.4892,-18.2449,-18.0239,-17.3703,-16.9162,-16.4522,-15.8001,-15.5758,-15.3043
15,-18.7031,-18.3225,-18.0371,-17.0784,-16.4264,-15.7905,-14.852,-14.5503,-14.2204
25,-18.5622,-18.1886,-17.9076,-16.8729,-16.1728,-15.481,-14.509,-14.1777,-13.8097


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,-0.0769,-0.069,-0.061,-0.0383,-0.0227,-0.0074,0.0146,0.0214,0.0301
5,-0.0407,-0.0307,-0.0228,0.0037,0.0216,0.0396,0.0658,0.0743,0.0845
15,-0.0016,0.0038,0.009,0.0264,0.0381,0.05,0.0678,0.0738,0.0795
25,0.0105,0.0148,0.0192,0.0323,0.0416,0.0511,0.0648,0.0693,0.0753


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,0.1144,0.126,0.1361,0.1634,0.1824,0.2023,0.2303,0.2397,0.2502
5,0.0163,0.0373,0.053,0.1055,0.1417,0.18,0.232,0.2503,0.2713
15,-0.0357,-0.0115,0.0116,0.0757,0.1209,0.1657,0.2271,0.2468,0.2692
25,-0.0385,-0.0138,0.0057,0.0719,0.119,0.1636,0.2254,0.2474,0.2694


In [93]:
real_equity_scenarios = reshape(scenarios[1, :, :] .+ scenarios[3, :,:], 1, 101, 10_000)
real_bond_scenarios = reshape(scenarios[2, :, :] .+ scenarios[3, :,:], 1, 101, 10_000)

ret_in_years = cum_returns_in_periods(vcat(real_equity_scenarios, real_bond_scenarios), periods, freq, true)
print_scenarios_summary(ret_in_years, assets_names[1:2], string.(periods))

for a in 1:2
    print_scenarios_percentiles(ret_in_years[a, :, :], [.01, 0.025, .05, .25, .5, .75, .95, .975, .99], string.(periods), string.(assets_names[a]))
end  



,Total_Stock_T_Bill,Price_10Y_T_Bill
1,-0.0123,0.0281
5,-0.0007,0.0165
15,0.0211,0.0142
25,0.0294,0.0128


,Total_Stock_T_Bill,Price_10Y_T_Bill
1,0.1284,0.0715
5,0.059,0.0308
15,0.0284,0.0126
25,0.0192,0.0091


,Total_Stock_T_Bill,Price_10Y_T_Bill
1,0.0282,0.0288
5,0.0215,-0.0433
15,0.0139,-0.0306
25,0.0186,0.0198


,Total_Stock_T_Bill,Price_10Y_T_Bill
1,-0.0884,0.0362
5,0.0099,-0.0173
15,-0.0723,0.0452
25,0.0557,-0.0221


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,-0.3114,-0.2602,-0.2229,-0.099,-0.013,0.0744,0.202,0.2431,0.2879
5,-0.136,-0.1153,-0.0975,-0.0408,-0.0012,0.039,0.0955,0.1147,0.1401
15,-0.0441,-0.0343,-0.0252,0.0017,0.021,0.0402,0.0687,0.0772,0.0858
25,-0.0148,-0.0081,-0.0023,0.0165,0.0293,0.0421,0.0614,0.0679,0.0747


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,-0.1363,-0.1112,-0.0889,-0.0205,0.0287,0.0752,0.1469,0.169,0.1953
5,-0.0552,-0.0434,-0.0348,-0.0045,0.0172,0.0374,0.066,0.0761,0.0876
15,-0.0152,-0.0108,-0.0067,0.0059,0.0142,0.0226,0.0348,0.0384,0.0428
25,-0.0083,-0.0048,-0.002,0.0067,0.0128,0.0188,0.0279,0.0308,0.0341


In [94]:
mdd, mddl = max_drawdown_and_length(real_equity_scenarios[1,:,:])

display(assets_names[1])
display("mdd")
display(quantile(mdd, [.03, .25, .5, .75, .97])')
display("mddl")
display(quantile(mddl, [.03, .25, .5, .75, .97])')

mdd, mddl = max_drawdown_and_length(real_bond_scenarios[1,:,:])

display(assets_names[2])
display("mdd")
display(quantile(mdd, [.03, .25, .5, .75, .97])')
display("mddl")
display(quantile(mddl, [.03, .25, .5, .75, .97])')

:Total_Stock_T_Bill

"mdd"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 0.264616  0.363991  0.431055  0.509521  0.64597

"mddl"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 18.0  30.0  40.0  56.0  88.0

:Price_10Y_T_Bill

"mdd"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 0.183664  0.253166  0.307027  0.372141  0.501246

"mddl"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 21.0  33.0  44.0  59.0  83.0